# GOPH 619 Lab1


### Question 1:
Express the number 1.481 in floating point binary format, as
discussed in class. Clearly show the binary strings representing the
exponent and the mantissa. At what digit does the “true error” in the
mantissa reach zero? Do you think that your result has zero round-off
error? Explain

In [130]:
######
#New attempt
import numpy as np
####Convert Decimal to mantissa and exponent
def decimal_2_mne(num):
    ########Convert decimal to base 2
    #Find if num is greater or less than 1
    n = 0
    while True:
        if num >1:
            num /= 2
            e = n+1
            n += 1
            #print(remainder,'if')
        else:
            num *= 2
            e = n-1
            n +=1
            #print(remainder,'else')
        if num>=0.5 and num <1:
            break
    print("Mantissa #:", num, "Exponent #:", e)
    return num, e

man, e = decimal_2_mne(1.481)


####Convert mantissa and exponent to binary
def me_2_bin(m,e):
    ####### Converts mantissa to binary
    rem = m
    sb = np.zeros(53)
    i = 0
    for k in range(-1,-53,-1):
        if rem >= 2**k:
            #print("remainder:", rem, "2^k:", 2**k, "k:",k)
            sb[i] = 1
            rem -= 2**k
            i +=1
        else:
            sb[i]= 0   
            i +=1
    ########
    #Convert exponent to binary
    rem = e
    se = np.zeros(10)
    i = 0
    for k in range(9,0,-1):
        if rem >= 2**k:
            se[i] = 1
            rem -= 2**k
            i +=1
        else:
            se[i] = 0
            i +=1
    print("Exponent:", se, "Mantissa:", sb)
    return sb,se

mantissa, exponent = me_2_bin(man,E)



Mantissa #: 0.7405 Exponent #: 1
Exponent: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] Mantissa: [1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0.
 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 1. 1. 0. 0. 0.]


In [ ]:
###

In [31]:
def float_2_bin(num):
    ####
    #Split number into integer part and decimal part
    import math
    dec,inte = math.modf(num)
    decimal = dec
    #####
    #Integer to binary
    bi = (bin(int(inte)))
    res = bi.removeprefix('0b')

    ######
    #Decminal to binary
    if decimal != 0.0:
        res2 =''
        n = 0
        while True:
            dec = dec * 2
            dec,i = math.modf(dec)
            i = int(i)
            res2 += str(i)
            if dec ==0:
                break
            n = n +1
        result = res + ':'+res2
    if decimal == 0.0:
        result = res
    return result

float_2_bin(2.5)


'10:1'

In [25]:
import math
math.frexp(2.5)

(0.625, 2)

### Question 2

Show the Taylor series for sin(x) for zero- through fifth-order.
Evaluate sin(x) for xi+1 = π/3 using a base point of xi = π/4 using the zero-
through fifth-order Taylor series expansions. Calculate the true and
approximate relative error for each estimate. Why is π/4 a convenient
base point for this function?

In [141]:


h = np.pi/3 - np.pi/4 # pi/12
x = np.pi/4
fun = math.sin(x)
der1 = math.cos(x)
der2 = -1*math.sin(x)
der3 = -1*math.cos(x)
der4 = math.sin(x)
der5 = math.cos(x)
der =[fun,der1,der2,der3,der4,der5]
true_answer = math.sin(np.pi/3)
ea = None
fact = 1
for i in range(6):
    if i ==0:
        sol = der[i]
        et = ((true_answer-sol)/(true_answer))*100
    if i !=0:
        oldsol =sol
        fact *=i
        sol = oldsol + (der[i]/fact)*h**i
        #sol = oldsol + (der[i]/math.factorial(i))*h**i
        et = ((true_answer-sol)/(true_answer))*100
        ea = abs((sol - oldsol) / sol)*100
        print(fact)
    print("solution:",sol,"True Error:",et,"Approximate Error:", ea)

math.sin(np.pi/3)


solution: 0.7071067811865475 True Error: 18.3503419072274 Approximate Error: None
1
solution: 0.8922269036098127 True Error: -3.0254885954703363 Approximate Error: 20.748099129750262
2
solution: 0.8679947362499555 True Error: -0.22739892581800955 Approximate Error: 2.791741279970057
6
solution: 0.8658800807233364 True Error: 0.016780461689357797 Approximate Error: 0.24422036881279358
24
solution: 0.8660184846039052 True Error: 0.0007989581487078311 Approximate Error: 0.01598163122719495
120
solution: 0.8660257314141455 True Error: -3.783141989694536e-05 Approximate Error: 0.0008367892520355209


0.8660254037844386

In [199]:
# def sin_sol(x,es,maxit):

#     n = 0
#     sol = x
#     ea = 100
#     while True:
#         if n == 0:
#             solold = sol
#             sol = (-1)**n * ((x**(2*n+1)) / (math.factorial(2*n+1)))
#             n +=1
#         if n >0:
#             solold = sol
#             sol = sol + (-1)**n * ((x**(2*n+1)) / (math.factorial(2*n+1)))
#             n +=1
#         if sol != 0:
#             ea = abs((sol - solold) / sol)*100
#             et = ((math.sin(x) - sol) / math.sin(x))*100
#         if ea <= es or n >= maxit:
#             break
#         print("sol =", sol, "et =", et, "ea =", ea)
#     v = sol    
#     return v, ea, et,n

# n = 5
# es = 0.5 * 10 **(2-n)
# sin_sol(np.pi/2, es, 10)

sol = 0.9248322292886504 et = 7.516777071134961 ea = 69.84662483087337
sol = 1.0045248555348174 et = -0.4524855534817407 ea = 7.933365292762019
sol = 0.9998431013994987 et = 0.015689860050127624 ea = 0.46824888112600327
sol = 1.0000035425842861 et = -0.0003542584286142514 ea = 0.01604406164130122


(0.999999943741051, 0.00035988434376645854, 5.625894905492146e-06, 6)

### Question 3

Generate sample values of ex from x = 0 to x = 5 using a step
size Δx = 0.5. Estimate the value of e2.4 using zero- through fourth-order
polynomial interpolation. Do this using the Vandermonde matrix,
Newton IP and Lagrange IP methods. Calculate the true and approximate
relative error for each estimate

In [148]:
dx = 0.5
X = np.arange(0,5,dx)
true_value = math.exp(2.4)

